In [1]:
import deeptrack as dt
import numpy as np

IMAGE_SIZE = 64
sequence_length = 10  # Number of frames per sequence
MIN_SIZE = 0.5e-6
MAX_SIZE = 1.5e-6
MAX_VEL = 10  # Maximum velocity. The higher the trickier!
MAX_PARTICLES = 3  # Max number of particles in each sequence. The higher the trickier!

# Defining properties of the particles
particle = dt.Sphere(
    intensity=lambda: 10 + 10 * np.random.rand(),
    radius=lambda: MIN_SIZE + np.random.rand() * (MAX_SIZE - MIN_SIZE),
    position=lambda: IMAGE_SIZE * np.random.rand(2),
    vel=lambda: MAX_VEL * np.random.rand(2),
    position_unit="pixel",
)

# Defining an update rule for the particle position
def get_position(previous_value, vel):

    newv = previous_value + vel
    for i in range(2):
        if newv[i] > 63:
            newv[i] = 63 - np.abs(newv[i] - 63)
            vel[i] = -vel[i]
        elif newv[i] < 0:
            newv[i] = np.abs(newv[i])
            vel[i] = -vel[i]
    return newv


particle = dt.Sequential(particle, position=get_position)

# Defining properties of the microscope
optics = dt.Fluorescence(
    NA=1,
    output_region=(0, 0, IMAGE_SIZE, IMAGE_SIZE),
    magnification=10,
    resolution=(1e-6, 1e-6, 1e-6),
    wavelength=633e-9,
)

# Combining everything into a dataset.
# Note that the sequences are flipped in different directions, so that each unique sequence defines
# in fact 8 sequences flipped in different directions, to speed up data generation
sequential_images = dt.Sequence(
    optics(particle ** (lambda: 1 + np.random.randint(MAX_PARTICLES))),
    sequence_length=sequence_length,
)
dataset = sequential_images >> dt.FlipUD() >> dt.FlipDiagonal() >> dt.FlipLR()

C:\Users\Maximilian\Documents\Chalmers\Advanced machine learning with neural networks\Advanced-ML-TIF360\hwc_venv\lib\site-packages\deeptrack\backend\_config.py:11: UserWarning: cupy not installed. GPU-accelerated simulations will not be possible
  warnings.warn(
C:\Users\Maximilian\Documents\Chalmers\Advanced machine learning with neural networks\Advanced-ML-TIF360\hwc_venv\lib\site-packages\deeptrack\backend\_config.py:25: UserWarning: cupy not installed, CPU acceleration not enabled
  warnings.warn("cupy not installed, CPU acceleration not enabled")
C:\Users\Maximilian\Documents\Chalmers\Advanced machine learning with neural networks\Advanced-ML-TIF360\hwc_venv\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other reposito

In [2]:
from tensorflow import keras
from tensorflow.keras import Model, layers, losses
import tensorflow as tf
Layer=keras.layers.Layer
Conv2D=keras.layers.Conv2D
MaxPool2D=keras.layers.MaxPooling2D
Dense=keras.layers.Dense
Flatten=keras.layers.Flatten
Reshape=keras.layers.Reshape

class Time2Vector(Layer): #Time embedding layer
    def __init__(self, seq_len, **kwargs):
        super(Time2Vector, self).__init__()
        self.seq_len = seq_len

    def build(self, input_shape):
        self.weights_linear = self.add_weight(name='weight_linear',
                                    shape=(int(self.seq_len),),
                                    initializer='uniform',
                                    trainable=True)

        self.bias_linear = self.add_weight(name='bias_linear',
                                    shape=(int(self.seq_len),),
                                    initializer='uniform',
                                    trainable=True)

        self.weights_periodic = self.add_weight(name='weight_periodic',
                                    shape=(int(self.seq_len),),
                                    initializer='uniform',
                                    trainable=True)

        self.bias_periodic = self.add_weight(name='bias_periodic',
                                    shape=(int(self.seq_len),),
                                    initializer='uniform',
                                    trainable=True)

    def call(self, x):
        x = tf.math.reduce_mean(x[:,:,:], axis=-1) # Convert (batch, seq_len, 5) to (batch, seq_len)
        time_linear = self.weights_linear * x + self.bias_linear
        time_linear = tf.expand_dims(time_linear, axis=-1) # (batch, seq_len, 1)

        time_periodic = tf.math.sin(tf.multiply(x, self.weights_periodic) + self.bias_periodic)
        time_periodic = tf.expand_dims(time_periodic, axis=-1) # (batch, seq_len, 1)
        return tf.concat([time_linear, time_periodic], axis=-1) # (batch, seq_len, 2)

In [3]:
class SingleAttention(Layer): #Attention layer
    def __init__(self, d_k, d_v):
        super(SingleAttention, self).__init__()
        self.d_k = d_k
        self.d_v = d_v

    def build(self, input_shape):
        self.query = Dense(self.d_k, input_shape=input_shape, kernel_initializer='glorot_uniform', bias_initializer='glorot_uniform')
        self.key = Dense(self.d_k, input_shape=input_shape, kernel_initializer='glorot_uniform', bias_initializer='glorot_uniform')
        self.value = Dense(self.d_v, input_shape=input_shape, kernel_initializer='glorot_uniform', bias_initializer='glorot_uniform')

    def call(self, inputs): # inputs = (in_seq, in_seq, in_seq)
        q = self.query(inputs[0])
        k = self.key(inputs[1])

        attn_weights = tf.matmul(q, k, transpose_b=True)
        attn_weights = tf.map_fn(lambda x: x/np.sqrt(self.d_k), attn_weights)
        attn_weights = tf.nn.softmax(attn_weights, axis=-1)

        v = self.value(inputs[2])
        attn_out = tf.matmul(attn_weights, v)
        return attn_out 

In [4]:
class MultiAttention(Layer):
    def __init__(self, d_k, d_v, h, d_f):
        super(MultiAttention, self).__init__()
        self.d_k = d_k
        self.d_v = d_v
        self.heads = h
        self.d_f = d_f
        self.attn_layers = []
        
    def build(self, input_shape):
        for head in range(self.heads):
            self.attn_layers.append(SingleAttention(self.d_k, self.d_v))
        self.dense = Dense(self.d_f, input_shape=input_shape, kernel_initializer='glorot_uniform', bias_initializer='glorot_uniform')
    
    def call(self, input):
        attention = [self.attn_layers[i](input) for i in range(self.heads)]
        conc_attention = tf.concat(attention, axis=1)
        mlp = self.linear(conc_attention)
        return mlp

In [5]:
class TransformerEncoder(Layer):
    def __init__(self, d_k, d_v, h, d_f, d_filt):
        super(TransformerEncoder, self).__init__()
        self.d_k = d_k
        self.d_v = d_v
        self.heads = h
        self.d_f = d_f
        self.d_filt = d_filt
    def build(self, input_shape):
        self.multi_head = MultiAttention(self.d_k, self.d_v, self.h, self_d_f)
        self.dropout = Dropout(rate=0.2)
        self.norm1 = LayerNormalization(epsilon=0.0001)
        self.conv1 = Conv2D(filters=self.d_f, kernel_size=1, activation='relu')
        self.conv2 = Conv2D(filters=self.d_filt, kernel_size=1)
        self.norm2 = LayerNormalization(epsilon=0.0001)
        
    
    def call(self, input):
        res = input[0]
        
        #Attention
        x = self.multi_head(input)
        x = self.dropout(x)
        x = self.norm1(x + res)
        
        #Feed-forward
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.norm2(x + res)
        
        return x

## Data processing

In [46]:
from tqdm import trange
def retrieve_data(data_size):
    frames = []

    for d in trange(data_size):
        video = dataset.update().resolve()
        for frame in video:
            frames.append(frame)

    return tf.stack(frames)

data_size = 1000

# Save the data
data = retrieve_data(data_size)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [09:26<00:00,  1.76it/s]


In [ ]:
np.save('video_data.npy', data)

In [48]:
# Load if data is already generated
data = np.load('video_data.npy')

#### Split into training and validation data

In [87]:
num_frames = len(data)
train_size = int(num_frames * 0.8)
val_size = num_frames - train_size

train_data = data[:train_size]
val_data = data[train_size:]

max_val = tf.reduce_max(tf.concat([train_data, val_data], axis=0))
train_data /= max_val
val_data /= max_val

print("Training data size:", train_data.shape)
print("Validation data size:", val_data.shape)

Training data size: (8000, 64, 64, 1)
Validation data size: (2000, 64, 64, 1)


#### Convolutional Autoencoder

In [86]:
Conv2D=keras.layers.Conv2D
MaxPool2D=keras.layers.MaxPooling2D
Dense=keras.layers.Dense
Flatten=keras.layers.Flatten
Reshape=keras.layers.Reshape
Input = keras.layers.Input
Sequential = keras.Sequential
Conv2DTranspose = keras.layers.Conv2DTranspose

k_size = 4
n_filters = 4
bottleneck_size = k_size**2*n_filters

class AutoEncoder(Model):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.encoder = Sequential([
            Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 1)),
            Conv2D(64, (4, 4), activation='relu', padding='same', strides=4),
            Conv2D(n_filters, (k_size, k_size), activation='relu', padding='same', strides=4),
            Flatten()
            ])
        
        bottleneck_size = k_size**2*n_filters
        self.decoder = Sequential([
            Input(shape=(bottleneck_size,)),
            Reshape(target_shape=(4, 4, n_filters)),
            Conv2DTranspose(n_filters, (4, 4), strides=(4, 4), activation='relu', padding='same'),
            Conv2DTranspose(64, (8, 8), strides=(4, 4), activation='relu', padding='same'),
            Conv2D(1, (1, 1), activation='linear', padding='same')
            ])
    def call(self, input):
        x = self.encoder(input)
        x = self.decoder(x)
        return x



autoencoder = AutoEncoder()
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

autoencoder.encoder.build(input_shape=(None, 64, 64, 1))
autoencoder.encoder.summary()

autoencoder.decoder.build(input_shape=(None, bottleneck_size))
autoencoder.decoder.summary()

Model: "sequential_46"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_69 (Conv2D)          (None, 16, 16, 64)        1088      
                                                                 
 conv2d_70 (Conv2D)          (None, 4, 4, 4)           4100      
                                                                 
 flatten_24 (Flatten)        (None, 64)                0         
                                                                 
Total params: 5,188
Trainable params: 5,188
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_47"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_23 (Reshape)        (None, 4, 4, 4)           0         
                                                                 
 conv2d_transpose_46 (Conv2D

In [88]:
autoencoder.fit(train_data, train_data,
                epochs=40,
                shuffle=True,
                validation_data=(val_data, val_data))

Epoch 1/40
250/250 [==============================] - 38s 149ms/step - loss: 0.0025 - val_loss: 5.1685e-04
Epoch 2/40
250/250 [==============================] - 37s 148ms/step - loss: 3.5882e-04 - val_loss: 2.6138e-04
Epoch 3/40
250/250 [==============================] - 37s 150ms/step - loss: 2.1049e-04 - val_loss: 1.8093e-04
Epoch 4/40
250/250 [==============================] - 38s 150ms/step - loss: 1.5653e-04 - val_loss: 1.4437e-04
Epoch 5/40
250/250 [==============================] - 37s 148ms/step - loss: 1.2953e-04 - val_loss: 1.2083e-04
Epoch 6/40
250/250 [==============================] - 37s 149ms/step - loss: 1.1358e-04 - val_loss: 1.1067e-04
Epoch 7/40
250/250 [==============================] - 37s 150ms/step - loss: 1.0172e-04 - val_loss: 9.9631e-05
Epoch 8/40
250/250 [==============================] - 37s 149ms/step - loss: 9.4361e-05 - val_loss: 9.5058e-05
Epoch 9/40
250/250 [==============================] - 38s 152ms/step - loss: 9.1515e-05 - val_loss: 8.4893e-05
Epoch

In [90]:
autoencoder.save('autoencoder_bottleneck_size='+str(bottleneck_size))

INFO:tensorflow:Assets written to: autoencoder_bottleneck_size=64\assets


INFO:tensorflow:Assets written to: autoencoder_bottleneck_size=64\assets
